In [1]:
# */10 * * * * cd ~/cron && /usr/local/bin/python3 cron_test.py >> ~/cron/cron.txt 2>&1

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from sendgrid import SendGridAPIClient
from dotenv import load_dotenv
from sendgrid.helpers.mail import Mail, Attachment, FileContent, FileName, FileType, Disposition
import time
import datetime
import pandas as pd
import os
import base64

def sendNotification():
    message = Mail(
        from_email='liao.675@osu.edu',
        to_emails='henry.huiliao@gmail.com',
        subject='TICKET PRICES',
        html_content='<strong>and easy to do anywhere, even with Python</strong>')
    
    with open('tickets.csv', 'rb') as f:
        data = f.read()
        f.close()
    encoded_file = base64.b64encode(data).decode()

    attachedFile = Attachment(
        FileContent(encoded_file),
        FileName('tickets.csv'),
        FileType('application/csv'),
        Disposition('attachment')
    )
    message.attachment = attachedFile
    
    try:
        sg = SendGridAPIClient(os.environ['SENDGRID_API_KEY'])
        response = sg.send(message)
        print(response.status_code)
        print(response.body)
        print(response.headers)
    except Exception as e:
        print(e.message)
        
load_dotenv('sendgrid.env')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

driver.get('https://www.google.com/travel/flights/search?tfs=CBwQAhoragwIAhIIL20vMDNoNjQSCjIwMjItMDctMDFyDQgDEgkvbS8wMTcyMzYoAHABggELCP___________wFAAUgDmAEC&tfu=EgYIARAAGAA&hl=zh-TW&gl=us&curr=USD')

driver.maximize_window()

buttons = driver.find_elements(by = By.CSS_SELECTOR, value = 'button.VfPpkd-LgbsSe.VfPpkd-LgbsSe-OWXEXe-Bz112c-M1Soyc.LjDxcd.XhPA0b.LQeN7.joofcc')

data = pd.DataFrame(columns = ['Date', 'Start Time', 'Landing Time', 'Airline', 'Price'])

startDate = driver.find_element(by = By.CSS_SELECTOR, value = 'input.RKk3r.eoY5cb.j0Ppje')
startDate = startDate.get_attribute("value")

month = int(startDate[0])

try:
    prevData = pd.read_csv('tickets.csv')
except:
    prevData = pd.DataFrame(columns = ['Date', 'Start Time', 'Landing Time', 'Airline', 'Price'])

update = False

try:
    day = int(startDate[2:3])
except:
    day = int(startDate[2])

year = datetime.date.today().year
    
if datetime.date.today().month > month:
    year += 1
    
startDate = datetime.datetime(year, month, day)

newMonth = False

while len(data) < 3 or month == startDate.month:
    try:
        time.sleep(2)
        WebDriverWait(driver, timeout = 1).until(lambda d: d.find_elements(by = By.CSS_SELECTOR, value = 'div.v6XMoe'))
        
        prices = driver.find_element(by = By.CSS_SELECTOR, value = 'div.v6XMoe')
        
        temp = prices.text.split()
        
        row = {'Date': startDate.strftime("%m/%d/%Y"), 'Start Time': temp[0], 'Landing Time': temp[2], 'Airline': temp[3], 'Price': temp[-1]}
        
        if not update:
            if not ((prevData['Start Time'] == temp[0]) & (prevData['Landing Time'] == temp[2]) & (prevData['Airline'] == temp[3]) & (prevData['Price'] == temp[-1]) & (prevData['Date'] == startDate.strftime("%m/%d/%Y"))).any():
                update = True

        data = data.append(row, ignore_index = True)
    except:
        if not update and startDate.strftime("%m/%d/%Y") in prevData['Date'].unique():
            update = True
        pass
    
    try:
        if month != startDate.month:
            month += 1
        
        buttons[1].click()
        startDate += datetime.timedelta(days=1)
    except:
        break

if update:
    data.to_csv('tickets.csv')
    sendNotification()
    
driver.close()
driver.quit()



====== WebDriver manager ======
Current google-chrome version is 104.0.5112
Get LATEST chromedriver version for 104.0.5112 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/104.0.5112.79/chromedriver_mac64.zip


In [4]:
prevData = pd.read_csv('tickets.csv')

In [5]:
prevData[1:5]

,Unnamed: 0,Date,Start Time,Landing Time,Airline,Price
1,1,09/29/2022,上午8:35,上午11:00,國泰航空,US$844
2,2,10/06/2022,上午8:35,上午11:00,國泰航空,US$844
3,3,10/13/2022,上午8:35,上午11:00,國泰航空,US$844
4,4,10/20/2022,上午8:35,上午11:00,國泰航空,US$844


In [7]:
startDate

datetime.datetime(2022, 11, 1, 0, 0)

In [3]:
data

,Date,Start Time,Landing Time,Airline,Price
0,09/22/2022,上午8:35,上午11:00,國泰航空,US$844
1,09/29/2022,上午8:35,上午11:00,國泰航空,US$844
2,10/06/2022,上午8:35,上午11:00,國泰航空,US$844
3,10/13/2022,上午8:35,上午11:00,國泰航空,US$844
4,10/20/2022,上午8:35,上午11:00,國泰航空,US$844
5,10/27/2022,上午8:35,上午11:00,國泰航空,US$844


# Calendar Method

In [5]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

driver.get('https://www.google.com/travel/flights/search?tfs=CBwQAhoragwIAhIIL20vMDNoNjQSCjIwMjItMDctMDFyDQgDEgkvbS8wMTcyMzYoAHABggELCP___________wFAAUgDmAEC&tfu=EgYIARAAGAA&hl=zh-TW&gl=us&curr=USD')

driver.maximize_window()

button = driver.find_element(by = By.CSS_SELECTOR, value = 'input.RKk3r.eoY5cb.j0Ppje')

button.click()

monthButtons = driver.find_elements(by = By.CSS_SELECTOR, value = 'button.VfPpkd-BIzmGd.SaBhMc.NNFoTc.Hkd4je.E9mvxc.V0XOz.a2rVxf.VfPpkd-BIzmGd-OWXEXe-yolsp')
action = ActionChains(driver)

driver.implicitly_wait(1.0)

calendar = driver.find_element(by = By.CSS_SELECTOR, value = 'div.Un8Fvc.Rzjj1.k8qXw')
action.move_to_element(calendar)
action.click(monthButtons[1])


# for _ in range(0, 6):
#     months = driver.find_elements(by = By.CSS_SELECTOR, value = 'div.lkvzbb.KQqAEc')
    
#     print(months[0].text)
#     print(months[1].text)
    
#     monthButtons[1].click()
#     monthButtons[1].click()



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/roryliao/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


IndexError: list index out of range

In [60]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

driver.get('https://www.google.com/travel/flights/search?tfs=CBwQAhoragwIAhIIL20vMDNoNjQSCjIwMjItMDctMDFyDQgDEgkvbS8wMTcyMzYoAHABggELCP___________wFAAUgDmAEC&tfu=EgYIARAAGAA&hl=zh-TW&gl=us&curr=USD')

driver.maximize_window()

calendarB = driver.find_element(by = By.CSS_SELECTOR, value = 'input.RKk3r.eoY5cb.j0Ppje')

calendarB.click()

WebDriverWait(driver, timeout = 10).until(lambda d: d.find_element(by = By.CSS_SELECTOR, value = 'div.JA2Doc.RNniQb'))

calendar = driver.find_element(by = By.CSS_SELECTOR, value = 'div.JA2Doc.RNniQb')

months = calendar.find_elements(by = By.CSS_SELECTOR, value = 'div.FyzlMe.ydXJud')

count = 1

for month in months:
    if count % 3 != 0:
        pairs = month.find_elements(by = By.CSS_SELECTOR, value = 'div.lkvzbb.KQqAEc')
        
        for pair in pairs:
            divs = pair.find_elements(by = By.TAG_NAME, value = 'div')
            
            for div in divs:
                try:
                    print(div.get_attribute('aria-label'))
                except:
                    pass
    else:
        monthButtons = driver.find_elements(by = By.CSS_SELECTOR, value = 'button.VfPpkd-BIzmGd.SaBhMc.NNFoTc.Hkd4je.E9mvxc.V0XOz.a2rVxf.VfPpkd-BIzmGd-OWXEXe-yolsp')
        ActionChains(driver).move_to_element(monthButtons[1]).click(monthButtons[1]).perform()
        ActionChains(driver).move_to_element(monthButtons[1]).click(monthButtons[1]).perform()
        
        WebDriverWait(driver, timeout = 10).until(EC.text_to_be_present_in_element((By.CLASS_NAME, "E2R7he.OhUk9.B1j46b.julK3b.RZ6mCd"), '$840'))
    
    count += 1;
        
driver.quit()



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_mac64.zip
Driver has been saved in cache [/Users/roryliao/.wdm/drivers/chromedriver/mac64/102.0.5005.61]


6月1日 週三
None
6月2日 週四
None
6月3日 週五
None
6月4日 週六
None
6月5日 週日
None
6月6日 週一
None
6月7日 週二
None
6月8日 週三
None
6月9日 週四
None
6月10日 週五
None
6月11日 週六
None
6月12日 週日
None
6月13日 週一
None
6月14日 週二
None
6月15日 週三
None
6月16日 週四
None
6月17日 週五
None
6月18日 週六
None
6月19日 週日
None
6月20日 週一
None
6月21日 週二
None
6月22日 週三
None
6月23日 週四
None
6月24日 週五
None
6月25日 週六
None
6月26日 週日
None
6月27日 週一
None
6月28日 週二
None
6月29日 週三
None
6月30日 週四
None
7月1日 週五 (出發日期)。
None
7月2日 週六
None
7月3日 週日
None
7月4日 週一
None
7月5日 週二
None
7月6日 週三
None
7月7日 週四
None
7月8日 週五
None
7月9日 週六
None
7月10日 週日
None
7月11日 週一
None
7月12日 週二
None
7月13日 週三
None
7月14日 週四
None
7月15日 週五
None
7月16日 週六
None
7月17日 週日
None
7月18日 週一
None
7月19日 週二
None
7月20日 週三
None
7月21日 週四
None
7月22日 週五
None
7月23日 週六
None
7月24日 週日
None
7月25日 週一
None
7月26日 週二
None
7月27日 週三
None
7月28日 週四
None
7月29日 週五
None
7月30日 週六
None
7月31日 週日
None


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x000000010dad4d19 chromedriver + 5197081
1   chromedriver                        0x000000010da62b23 chromedriver + 4729635
2   chromedriver                        0x000000010d6402b8 chromedriver + 393912
3   chromedriver                        0x000000010d6752fd chromedriver + 611069
4   chromedriver                        0x000000010d675581 chromedriver + 611713
5   chromedriver                        0x000000010d6a75a4 chromedriver + 816548
6   chromedriver                        0x000000010d69282d chromedriver + 731181
7   chromedriver                        0x000000010d6a52a1 chromedriver + 807585
8   chromedriver                        0x000000010d6926f3 chromedriver + 730867
9   chromedriver                        0x000000010d668a49 chromedriver + 559689
10  chromedriver                        0x000000010d669a75 chromedriver + 563829
11  chromedriver                        0x000000010daa6bdd chromedr